In [ ]:
# Define custom dataset
class FaceDataset(Dataset):
    def __init__(self, df,df2, train, num_sample = None, transform = None, num_img_pool = 10, ):
        # set random seed for FaceDataset
        np.random.seed(seed)
        random.seed(seed)
        # create constructors
        self.unique_img_name = None
        self.train = train
        self.data = dict()
        self.images = list()
        self.images2 = list()
        self.identities = list()
        self.identities2 = list()
        # label to indices
        # self.label_to_indices = dict()
        self.labels = list()
        self.labels2 = list()
        self.img_name = list()
        self.img_name2 = list()
        # read csv file
        self.df = df
        self.df2 = df2
        # set the transformation
        self.transform = transform
        # drop last n row from dataframe
        self.df = self.df.head(num_sample)
        #get the length of entire dataset
        self.len_ = len(self.df)
        if num_sample is None or num_sample > self.len_:
            num_sample = self.len_
        # load imgs
        self.load_imgs(self.df, self.df2, num_imgs = num_img_pool, max = num_sample)
        self.random_list = list()

    def __len__(self):
        return self.len_

    def random_int(self):
        max = len(self.images)
        while True:
            rand_idx = np.random.randint(0,max)
            if not rand_idx in self.random_list:
                self.random_list.append(rand_idx)
                break
            if len(self.random_list) == max:
                self.random_list = list()
                print('All included')
                break
            # print('x',rand_idx)
            # print('temp:',self.random_list)
        return rand_idx

    def __getitem__(self, idx):
        if self.train:
            anchor_img = self.images[idx]
            anchor_label = self.labels[idx]
            pos_idx = np.random.choice(np.arange(len(self.images))[self.labels == anchor_label])
            neg_idx = np.random.choice(np.arange(len(self.images))[self.labels != anchor_label])

            pos_img = self.images[pos_idx]
            neg_img = self.images[neg_idx]

            pos_label = self.labels[pos_idx]
            neg_label = self.labels[neg_idx]

            if self.transform is None:
                img_to_tensor = transforms.ToTensor()
                anchor_img = img_to_tensor(anchor_img)
                pos_img = img_to_tensor(pos_img)
                neg_img = img_to_tensor(neg_img)
            else:
                anchor_img = self.transform(anchor_img)
                pos_img = self.transform(pos_img)
                neg_img = self.transform(neg_img)
            return anchor_img, pos_img, neg_img

        else:

            idx = self.random_int()

            label = False
            anchor_img = self.images[idx]
            anchor_label = self.labels[idx]
            if idx % 2 == 0:
                test_idx = np.random.choice(np.arange(len(self.images2))[self.labels == anchor_label])
                label = True

            else:
                test_idx = np.random.choice(np.arange(len(self.images2))[self.labels != anchor_label])

            neg_idx = np.random.choice(np.arange(len(self.images))[self.labels != anchor_label])
            test_img = self.images2[test_idx]
            neg_img = self.images2[neg_idx]

            if self.transform is None:
                img_to_tensor = transforms.ToTensor()
                anchor_img = img_to_tensor(anchor_img)
                test_img = img_to_tensor(test_img)
                neg_img = img_to_tensor(neg_img)

            else:
                anchor_img = self.transform(anchor_img)
                test_img = self.transform(test_img)
                neg_img = self.transform(neg_img)

            return anchor_img, test_img, neg_img ,label, self.identities[idx], self.img_name[idx], self.identities2[test_idx], self.img_name2[test_idx]



    def face_regs(self, path: str):
        try:
            image = face_recognition.load_image_file(path)
            face_locations = face_recognition.face_locations(image)

            # print("I found {} face(s) in this photograph.".format(len(face_locations)))

            for face_location in face_locations:

                # Print the location of each face in this image
                top, right, bottom, left = face_location
                #     print("A face is located at pixel location Top: {}, Left: {}, Bottom: {}, Right: {}".format(top, left, bottom, right))

                # You can access the actual face itself like this:
                face_image = image[top:bottom, left:right]
                pil_image = Image.fromarray(face_image).convert('RGB')
            #             plt.imshow(pil_image)
            #                 print(pil_image)
            return (True, pil_image)
        except:
            return (False, None)

            # load imgs from pandas to memory and define the maximum number of images
    def load_imgs(self, df, df2, num_imgs, max):
        # iterate thought each row
        count_row = 0
        for i, row in tqdm(df.iterrows(), total = max):
            # get identity of each row
            row_identity = row['identity']
            row_identity2 = df2.iloc[count_row]

            # append each identity to numberical value
            # self.label_to_indices[int(row_identity)] = i
            count_img = 0
            # loop imgs in each identity
            for index, img_name in enumerate(row['path']):
                if count_img > num_imgs:
                    break
                # concatenate the directoru and image name
                path_to_image = img_name
                path_to_image2 = np.random.choice(row_identity2['path'])

                # open image and convert to RGB
                img = Image.open(path_to_image).convert('RGB')
                img2 = Image.open(path_to_image2).convert('RGB')
                self.labels.append(i)
                self.images.append(img)
                self.img_name.append(path_to_image)
                self.identities.append(str(row_identity))

                self.labels2.append(i)
                self.images2.append(img2)
                self.img_name2.append(path_to_image2)
                self.identities2.append(str(row_identity2))

                count_img += 1  # print('Added img '+ str(row_identity))
            count_row +=1

        self.labels = np.array(self.labels)
    def return_val(self):
        return self.labels

In [1]:
import numpy as np

In [3]:
'1' == np.array(['1','2','3'])

array([ True, False, False])